In [0]:
#Preprocess data
import pandas as pd
from sklearn.feature_extraction import text
import nltk
from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# fake is 0, true is 1
#load the data using pandas
x_train = pd.read_csv("x_train.csv", index_col=[0])
x_val = pd.read_csv("x_val.csv", index_col=[0])

#lowercase
x_train["content"] = x_train["content"].str.lower()
x_val["content"] = x_val["content"].str.lower()

#remove puctuation and stop word
stop = text.ENGLISH_STOP_WORDS
x_train["content"] = x_train["content"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
x_val["content"] = x_val["content"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


x_train["content"] = x_train["content"].str.replace('[^\w\s]','')
x_val["content"] = x_val["content"].str.replace('[^\w\s]','')



In [0]:
#stemming process
#stemmer reference: https://stackoverflow.com/questions/43795310/apply-porters-stemmer-to-a-pandas-column-for-each-word

# porter_stemmer = PorterStemmer()

# def stem(sentence):
#     tokens = sentence.split()
#     stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
#     return ' '.join(stemmed_tokens)


# x_train["content"] = x_train["content"].apply(stem)
# x_val["content"] = x_val["content"].apply(stem)


In [0]:
import nltk as nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
#lemmatization\
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

def lemma(sentence):
    tokens = sentence.split()
    lemmaed_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmaed_tokens)


x_train["content"] = x_train["content"].apply(lemma)
x_val["content"] = x_val["content"].apply(lemma)

xx_train = x_train["content"]
xx_val = x_val["content"]



In [0]:
x_val["content"]

0       moody downgrade uk rating brexit growth fearsl...
1       damaged new zealand fuel pipeline restarted su...
2       china say ban petroleum export north koreashan...
3       u challenged rising north korea tension russia...
4       temper fray search survivor wind mexico quakem...
                              ...                        
4485    mcpain john mccain furious iran treated sailor...
4486    justice yahoo settle email privacy classaction...
4487    sunnistan allied safe zone plan territorial bo...
4488    blow 700 million al jazeera america finally ca...
4489    10 u navy sailor held iranian military sign ne...
Name: content, Length: 4490, dtype: object

In [0]:
#tokenizer and padding
tokenizer = Tokenizer(num_words=50000, oov_token=None)
tokenizer.fit_on_texts(xx_train)
word_index = tokenizer.word_index
xx_train = tokenizer.texts_to_sequences(xx_train) 
xx_val = tokenizer.texts_to_sequences(xx_val)
maxlen = max([len(x) for x in xx_train]) 

xx_train = pad_sequences(xx_train, maxlen=maxlen)
xx_val = pad_sequences(xx_val, maxlen=maxlen)

In [0]:
from keras.layers import Activation, Input, Embedding, Reshape, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers
from keras.layers import Embedding
import os
from keras import regularizers
from keras import Sequential
from keras.layers import MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization


#reference of cnn: https://www.kaggle.com/au1206/text-classification-using-cnn
#reference: https://keras.io/examples/imdb_cnn/
# we have xx_train, y_train, xx_val, y_val
y_train =  pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")
y_train = y_train["label"]
y_val = y_val["label"]
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)


batch_size = 32
embedding_dims = 100
filters = 32
kernel_size = 5

epochs = 50

embedding_layer = Embedding(50001, 100,input_length=maxlen)


model = Sequential()
#model.add(Reshape([100], input_shape=[100]))
model.add(embedding_layer)
#model.add(Reshape((100,100)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters, kernel_size, padding='valid', activation='elu', activity_regularizer=regularizers.l2(0.01)))
model.add(MaxPooling1D())
#model.add(Conv1D(filters, kernel_size, padding='valid', activation='elu', activity_regularizer=regularizers.l2(0.01)))
#model.add(MaxPooling1D())

model.add(Flatten())

model.add(Dense(2, activation='elu'))

model.add(Dropout(0.2))




model.add(Dense(2, activation='sigmoid'))
adam = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#model.fit(xx_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(xx_val, y_val), shuffle=True)

#model.summary()

Using TensorFlow backend.


In [0]:
model.fit(xx_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(xx_val, y_val), shuffle=True)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40408 samples, validate on 4490 samples
Epoch 1/50
40408/40408 [==============================] - 38s 930us/step - loss: 76.8302 - accuracy: 0.5265 - val_loss: 2.8469 - val_accuracy: 0.4938
Epoch 2/50
40408/40408 [==============================] - 31s 768us/step - loss: 2.0914 - accuracy: 0.6043 - val_loss: 2.5011 - val_accuracy: 0.9003
Epoch 3/50
40408/40408 [==============================] - 31s 767us/step - loss: 1.3400 - accuracy: 0.8944 - val_loss: 0.9709 - val_accuracy: 0.9244
Epoch 4/50
40408/40408 [==============================] - 31s 769us/step - loss: 0.7649 - accuracy: 0.9190 - val_loss: 0.5654 - val_accuracy: 0.9532
Epoch 5/50
40408/40408 [==============================] - 31s 773us/step - loss: 0.5561 - accuracy: 0.9213 - val_loss: 0.7903 - val_accuracy: 0.8546
Epoch 6/50
40408/40408 [==============================] - 31s 766us/step - loss: 0.5693 - accuracy: 0.9226 - val_loss: 0.5441 - val_accuracy: 0.9606
Epoch 7/50
40408/40408 [==============================] 

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4311, 100)         5000100   
_________________________________________________________________
batch_normalization_1 (Batch (None, 4311, 100)         400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 4311, 100)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4307, 32)          16032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2153, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 68896)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                

In [0]:
from sklearn.metrics import confusion_matrix
prediction = model.predict(xx_val)


c = confusion_matrix(y_val.argmax(axis=1), prediction.argmax(axis=1))
print(c)

[[2156  192]
 [  12 2130]]


In [0]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix  
y_pred = np.argmax(prediction, axis=1) 

print(precision_score(y_val.argmax(axis=1), y_pred , average="macro")) 
print(recall_score(y_val.argmax(axis=1), y_pred , average="macro")) 
print(f1_score(y_val.argmax(axis=1), y_pred , average="macro"))



0.9558888030740773
0.9563130192451768
0.9545641780241341


In [0]:
xx_val.shape

In [0]:
y_val.shape

In [0]:
xx_val

In [0]:
#start to prepare postprocessing data
# reference: https://stackoverflow.com/questions/38674027/find-the-row-indexes-of-several-values-in-a-numpy-array
import csv
false_neg = open('false_neg.csv', 'w')
false_list1 = xx_val[(y_val.argmax(axis=1) == 1) & (prediction.argmax(axis=1) == 0)] #False Negatives
wr2 = csv.writer(false_neg, quoting=csv.QUOTE_ALL)

out = np.where((xx_val==false_list1[:,None]).all(-1))[1]
print(out)
wr2.writerow(['index'])
for x in out:
  wr2.writerow([x])


# for x in false_list1:
#     print(x)
#     wr2.writerow([x])

In [0]:
import csv
#import numpy_indexed as npi
false_pos = open('false_pos.csv', 'w')
false_list2 = xx_val[(y_val.argmax(axis=1) == 0) & (prediction.argmax(axis=1) == 1)] #False Positives
wr3 = csv.writer(false_pos, quoting=csv.QUOTE_ALL)

# dims = xx_val.max(0)+1
# out2 = np.where(np.in1d(np.ravel_multi_index(xx_val.T,dims),
#                         np.ravel_multi_index(false_list2.T,dims)))[0]

#result = npi.indices(xx_val, searched_values)
# dims = xx_val.max(0)+1
# X1D = np.ravel_multi_index(xx_val.T,dims)
# searched_valuesID = np.ravel_multi_index(false_list2.T,dims)
# sidx = X1D.argsort()
# out2 = sidx[np.searchsorted(X1D,searched_valuesID,sorter=sidx)]
#out2 = np.where((xx_val==false_list2[:,None]).all(-1))[1]
result = [[i for i,row in enumerate(xx_val) if (s==row).all()] for s in false_list2]
for x in result:
  if(len(x)==2):
    del x[1]
wr3.writerow(['index'])
for x in result:
   wr3.writerow(x)